In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# ========================================
# 1. CSV 읽기
# ========================================
csv_path = 'D:/emergency_room/data-analysis/data/simulated/emergency_predictions.csv'
df = pd.read_csv(csv_path)

print(f"✅ CSV 로드 완료: {len(df)}개 데이터")
print(f"\n데이터 샘플:\n{df.head()}")

# ========================================
# 2. MySQL 연결
# ========================================
try:
    connection = mysql.connector.connect(
        host='localhost',
        port=3306,
        database='emergency_room',
        user='root',
        password='1234'  # ← 🔑 비밀번호 입력!
    )
    
    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f"\n✅ MySQL 연결 성공! (버전: {db_info})")
        
        cursor = connection.cursor()
        
        # ========================================
        # 3. 기존 데이터 삭제 (중복 방지)
        # ========================================
        cursor.execute("DELETE FROM emergency_predictions")
        connection.commit()
        print(f"기존 데이터 삭제 완료")
        
        # ========================================
        # 4. 데이터 삽입
        # ========================================
        insert_query = """
        INSERT INTO emergency_predictions 
        (hospital_id, prediction_hour, predicted_wait_time, congestion_level, congestion_color)
        VALUES (%s, %s, %s, %s, %s)
        """
        
        # DataFrame을 리스트로 변환
        data_list = df.values.tolist()
        
        print(f"\n데이터 삽입 시작...")
        
        # 배치 삽입 (한 번에 모두 삽입)
        cursor.executemany(insert_query, data_list)
        connection.commit()
        
        print(f"✅ {cursor.rowcount}개 데이터 삽입 완료!")
        
        # ========================================
        # 5. 확인
        # ========================================
        cursor.execute("SELECT COUNT(*) FROM emergency_predictions")
        total_count = cursor.fetchone()[0]
        print(f"\n📊 테이블 총 레코드 수: {total_count}")
        
        # 혼잡도별 분포 확인
        cursor.execute("""
            SELECT congestion_level, COUNT(*) as count
            FROM emergency_predictions
            GROUP BY congestion_level
        """)
        
        print(f"\n혼잡도 분포:")
        for row in cursor.fetchall():
            print(f"  {row[0]}: {row[1]}개")
        
        # 샘플 데이터 확인
        cursor.execute("SELECT * FROM emergency_predictions LIMIT 5")
        print(f"\n샘플 데이터:")
        for row in cursor.fetchall():
            print(f"  {row}")
        
        cursor.close()
        print(f"\n✅ 모든 작업 완료!")

except Error as e:
    print(f"❌ MySQL 오류 발생: {e}")
    
except Exception as e:
    print(f"❌ 오류 발생: {e}")

finally:
    if connection.is_connected():
        connection.close()
        print(f"\nMySQL 연결 종료")

✅ CSV 로드 완료: 1104개 데이터

데이터 샘플:
  hospital_id  prediction_hour  predicted_wait_time congestion_level  \
0       ER001                0                    9               여유   
1       ER001                1                    9               여유   
2       ER001                2                    6               여유   
3       ER001                3                    6               여유   
4       ER001                4                    5               여유   

  congestion_color  
0            green  
1            green  
2            green  
3            green  
4            green  

✅ MySQL 연결 성공! (버전: 8.0.42)
기존 데이터 삭제 완료

데이터 삽입 시작...
✅ 1104개 데이터 삽입 완료!

📊 테이블 총 레코드 수: 1104

혼잡도 분포:
  여유: 493개
  보통: 577개
  혼잡: 34개

샘플 데이터:
  (1, 'ER001', 0, 9, '여유', 'green', datetime.datetime(2025, 10, 13, 12, 24, 21))
  (2, 'ER001', 1, 9, '여유', 'green', datetime.datetime(2025, 10, 13, 12, 24, 21))
  (3, 'ER001', 2, 6, '여유', 'green', datetime.datetime(2025, 10, 13, 12, 24, 21))
  (4, 'ER001', 3, 6,

C:\Users\yj-603-12\AppData\Local\Temp\ipykernel_18948\1139218056.py:27: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()
